In [1]:
import torch
import numpy as np
import os
import shutil
from datetime import datetime
import math
from math import exp
from random import seed
import random as rand
import matplotlib.pyplot as plt
import pickle 

import sys
sys.path.append('../common/')
from utils import *
from optimisation import *
from loo_cv import *
from rbf_tools import *
from two_dim import *

In [2]:
seed=0
torch.manual_seed(seed)

In [3]:
import torch.autograd as autograd         # computation graph
from torch import Tensor                  # tensor node in the computation graph
import torch.nn as nn                     # neural networks
import torch.nn.functional as F           # layers, activations and more
import torch.optim as optim               # optimizers e.g. gradient descent, ADAM, etc.
from torch.jit import script, trace       # hybrid frontend decorator and tracing jit
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR

In [4]:
class Net(nn.Module):
    def __init__(self,N=10):
            super(Net, self).__init__()
            
            nInputs = int(N*N/2-N/2)
            
            self.linear_relu_stack = nn.Sequential(
                nn.Linear(in_features=nInputs, out_features=64),
                nn.ReLU(),
                nn.Linear(in_features=64, out_features=64),
                nn.ReLU(),
                nn.Linear(in_features=64, out_features=64),
                nn.ReLU(),
                nn.Linear(in_features=64, out_features=32),
                nn.ReLU(),
                nn.Linear(in_features=32, out_features=16),
                nn.ReLU(),
                nn.Linear(in_features=16, out_features=1)
            )
            # map to positive
            
            
    def forward(self, x):
            logits = self.linear_relu_stack(x)
            return logits

# generate features

In [5]:
def upper_half(A):
    n = A.shape[0]
    return 1/A[np.triu_indices(n, k = 1)]

In [6]:
# network to be tested:
directory = "plots/"
if not os.path.exists(directory):
    os.makedirs(directory)

n=10
path =  '../network/results_sorted_1d_2d/best_model.pt'
model=Net()
model.load_state_dict(torch.load(path))
model.eval()

Net(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=45, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=64, bias=True)
    (5): ReLU()
    (6): Linear(in_features=64, out_features=32, bias=True)
    (7): ReLU()
    (8): Linear(in_features=32, out_features=16, bias=True)
    (9): ReLU()
    (10): Linear(in_features=16, out_features=1, bias=True)
  )
)

# interpolation two dimension 

In [7]:
import math
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import os
import shutil
from datetime import datetime
import math
from math import exp
from random import seed
from scipy.spatial import distance
import pickle

def phi(f, x, y):
    z = (1 + (f * np.linalg.norm(x-y)) ** 2) ** (-0.5)
    return z

In [8]:
def euclidean_distance(p1, p2):
    return np.sqrt(np.sum((p1 - p2) ** 2))

def find_nearest_neighbors(points):
    num_points = points.shape[0]
    distances = np.full(num_points, np.inf)
    
    for i in range(num_points):
        for j in range(num_points):
            if i != j:
                distance = euclidean_distance(points[i], points[j])
                if distance < distances[i]:
                    distances[i] = distance
    
    return distances
# Function to compute the Euclidean distance between two points
def dist(p1, p2):
    return math.hypot(p1[0] - p2[0], p1[1] - p2[1])

# Function to check if a point is inside a circle
def is_inside_circle(p, c):
    return dist(p, c[:2]) <= c[2]

# Function to compute the circle from 3 points
def circle_from_3_points(p1, p2, p3):
    A = p2[0] - p1[0]
    B = p2[1] - p1[1]
    C = p3[0] - p1[0]
    D = p3[1] - p1[1]
    E = A * (p1[0] + p2[0]) + B * (p1[1] + p2[1])
    F = C * (p1[0] + p3[0]) + D * (p1[1] + p3[1])
    G = 2 * (A * (p3[1] - p2[1]) - B * (p3[0] - p2[0]))
    
    if G == 0:  # Collinear points
        return None
    
    cx = (D * E - B * F) / G
    cy = (A * F - C * E) / G
    r = dist((cx, cy), p1)
    
    return (cx, cy, r)

# Function to compute the circle from 2 points
def circle_from_2_points(p1, p2):
    cx = (p1[0] + p2[0]) / 2
    cy = (p1[1] + p2[1]) / 2
    r = dist(p1, p2) / 2
    return (cx, cy, r)

# Recursive function to find the minimum enclosing circle
def welzl(P, R, n):
    if n == 0 or len(R) == 3:
        if len(R) == 0:
            return (0, 0, 0)
        elif len(R) == 1:
            return (R[0][0], R[0][1], 0)
        elif len(R) == 2:
            return circle_from_2_points(R[0], R[1])
        else:
            return circle_from_3_points(R[0], R[1], R[2])
    
    idx = rand.randint(0, n - 1)
    p = P[idx]
    P[idx], P[n - 1] = P[n - 1], P[idx]
    
    d = welzl(P, R, n - 1)
    
    if is_inside_circle(p, d):
        return d
    
    return welzl(P, R + [p], n - 1)

# Function to find the minimum enclosing circle
def find_min_circle(points):
    P = points[:]
    #random.shuffle(P)
    return welzl(P, [], len(P))

In [9]:
from sklearn.metrics.pairwise import euclidean_distances
from scipy.spatial.distance import cdist
import time

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import map_coordinates
from skimage.transform import resize

# Create a synthetic checkerboard image
def create_checkerboard(size=(200, 200), num_checks=10):
    # Create a checkerboard pattern
    x = np.linspace(0, num_checks, size[1])
    y = np.linspace(0, num_checks, size[0])
    X, Y = np.meshgrid(x, y)
    checkerboard = ((np.floor(X) + np.floor(Y)) % 2)
    return checkerboard


def upscale_image(image, downscaled_image):
    original_shape = image.shape

    # Prepare data for RBF interpolation
    # interpolation points
    x = np.linspace(0, original_shape[0] - 1, downscaled_image.shape[0])
    y = np.linspace(0, original_shape[1] - 1, downscaled_image.shape[1])
    xv, yv = np.meshgrid(x, y)
    coords = np.vstack([xv.ravel(), yv.ravel()]).T  # Create a list of coordinates
    
    values = downscaled_image.ravel()  # Flatten grayscale values

    #evaluation points
    x_full = np.linspace(0, original_shape[0] - 1, original_shape[0])
    y_full = np.linspace(0, original_shape[1] - 1, original_shape[1])
    xv_full, yv_full = np.meshgrid(x_full, y_full)
    coords_full = np.vstack([xv_full.ravel(), yv_full.ravel()]).T

    p_ref=np.array([0,0])    
    no_evaluation_points=coords_full.shape[0]
    distance =cdist(coords_full,coords)
    closest = np.argpartition(distance, 10, axis=1)
    scaled_training_set_flat = []
    eps_final_hardy=np.zeros((no_evaluation_points,1))
    eps_final_franke=np.zeros((no_evaluation_points,1))
    eps_final_mfranke=np.zeros((no_evaluation_points,1))
    eps_final_rippa=np.zeros((no_evaluation_points,1))
    
    x_coords = []
    f=[]
    upscaled_flat=np.zeros((5,no_evaluation_points))
    
    for i in range(no_evaluation_points):
        x_local=np.zeros((10,2))
        fpart=np.zeros((10+1,1))
        for j in range(10):
            x_local[j]=coords[closest[i][j]]
            fpart[j]=values[closest[i][j]]
        x_local = sorted( x_local, key = lambda x: np.linalg.norm(x - p_ref ) )
        x_local = np.reshape(x_local, (10,2)) 
        x_coords.append(x_local)
        f.append(fpart)
        x_axis=x_local.reshape(10,2)
        # Hardy approach
        nearest_distances = find_nearest_neighbors(x_axis)
        d=np.sum(nearest_distances)/n        
        eps_final_hardy[i]=1/(0.815*d)

        # Franke approach
        x_axis_list=x_axis.tolist()        
        circle = find_min_circle(x_axis_list)
        eps_final_franke[i]=0.8*(10**(1/2))/(2 * circle[2])

        # Modified Franke approach
        eps_final_mfranke[i]=0.8*(10**(1/4))/(2 * circle[2])

        # Rippa approach       
        rhs_rippa = fpart[:10]
        eps_v = [0.001, 0.002, 0.005, 0.0075, 0.01, 0.02, 0.05, 0.075, 0.1, 0.2, 0.5, 0.75, 1.0,\
                 2.0, 5.0, 7.5, 10.0, 20.0, 50.0, 75.0, 100.0, 200.0, 500.0, 1000.0] # from python library
        best_eps = 0
        old_error = np.inf        
        for eps in eps_v:
            error_r = rippa_cv(eps, x_axis, rhs_rippa)
            if error_r < old_error:
               best_eps = eps
               old_error = error_r
        eps_final_rippa[i]=best_eps
        
        # NN approach
        xxx=generate_distance_from_coordinates(x_axis)               
        training_set_distances_flatten = upper_half(xxx)
        scaled_training_set_flat.append(training_set_distances_flatten)

    scaled_training_set_tensor = torch.tensor(scaled_training_set_flat,dtype=torch.float)
    scaled_training_set_tensor.reshape((no_evaluation_points,int(10*10/2-10/2)))
        
    eps_final= model(scaled_training_set_tensor)
    eps_final_optimisation = eps_final.detach().numpy()

    for j in range(no_evaluation_points):
        for count,eps in enumerate([eps_final_rippa[j][0],eps_final_hardy[j][0],eps_final_franke[j][0],eps_final_mfranke[j][0],eps_final_optimisation[j][0]]):
            L =  get_int_matrix(x_coords[j],eps)                
            fx=f[j]
           
            w=np.linalg.solve(L, fx)             
            
            s=np.zeros((1,10+1))
            for i in range(10):      
                s[0,i] = phi(eps,coords_full[j] , x_coords[j][i])
            s[0,-1]=1.0        
            
            upscaled_flat[count][j]=np.matmul(s,w)          
        
    upscaled_image=[]
    for count in range(5):        
        upscaled_image.append(upscaled_flat[count].reshape(original_shape[:2]))
    return upscaled_image

# Visualize the images
def display_images(upscaled_image,file_name):
    
    fig = plt.figure(figsize=(10, 6))
    plt.imshow(upscaled_image, cmap='gray')
    plt.axis('off')
    plt.savefig(f'upscaled_'+file_name,bbox_inches='tight', dpi=150)
    plt.close(fig)


image = create_checkerboard()  # Create a synthetic checkerboard image
original_shape = image.shape

downscale_factor = 0.5
downscaled_image = resize(image, (int(original_shape[0] * downscale_factor),
                                  int(original_shape[1] * downscale_factor)))

upscaled_image = upscale_image(image, downscaled_image)  # upscale the image

fig1 = plt.figure(figsize=(10, 6))
plt.imshow(image, cmap='gray')
plt.axis('off')
plt.savefig(f'original',bbox_inches='tight', dpi=150)
plt.close(fig1)

fig2 = plt.figure(figsize=(5, 3))
plt.imshow(downscaled_image, cmap='gray')
plt.axis('off')
plt.savefig(f'downscaled',bbox_inches='tight', dpi=150)
plt.close(fig2)

strategies=['Rippa','Hardy','Franke','Modified Franke', 'NN']

mse=[0 for i in range(5)]
psnr=[0 for i in range(5)]

for count in range(5):
    display_images(upscaled_image[count],strategies[count])  # Display the images
    mse[count] = np.mean((image - upscaled_image[count]) ** 2)
    psnr[count]=20*np.log10(np.max(image)/np.sqrt(mse[count]))

print('mse',mse)
print('psnr',psnr)
